In [666]:
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import numpy as np

# Auth
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('gspread_creds.json', scope)
client = gspread.authorize(creds)

## Cleaning

#### Definition

In [667]:
sources = [
    'TFR-500', 
    'Tenenbaum',
    'Moodys 32',
    'Moodys 34',
    'IDed in HB 32', 
    'IDed in HB 34',
    'IDed in HB GmbH'
]

In [668]:
# TFR-500 mapping

org_type_mapping = {
    'Corporation (Aktiengesellschaft)': 'AG',
    'GmbH': 'GmbH',
}
link_type_mapping = {
    'ownership': 'USO', 
    'unknown': 'USOP', 
    'Partnership': 'USC', 
    'branch': 'USO'
}

In [669]:
# Tenenbaum, Moodys 32, 34 mapping

def assign_link_type(row):
    if row['mentions a subsidiary']:
        link_type = 'USO'
    elif row['mentions stock ownership']:
        link_type = 'USOP'
    elif row['mentions an affiliated company']:
        link_type = 'USC'
    elif row['other types of agreement']:
        link_type = 'USC'
    elif row['is a subsidiary of a German firm']:
        link_type = 'DEO'
    else:
        link_type = 'Others'

    return link_type

def assign_org_type(row):
    if row['affiliated company is AG']:
        org_type = 'AG'
    elif row['affiliated company is GmbH']:
        org_type = 'GmbH'
    else:
        org_type = 'Others'

    return org_type

In [670]:
# HB AG mapping

def assign_link_type_HB(row):
    if row['Grossaktionär is a US comp']:
        link_type = 'USO'
    elif row['Director attached to a US company']:
        link_type = 'USOP'
    elif row['Other indicators are strong']:
        link_type = 'USOP'
    elif row['US location linked to a director']:
        link_type = 'USC'
    elif row['Bond issuance in the US']:
        link_type = 'USB'
    else:
        link_type = 'Others'

    return link_type

### TFR-500

In [671]:
# Open the sheet
worksheet = client.open("tfr500_summarized").worksheet("classified")

# Get all records as list of dicts
records = worksheet.get_all_records()

# Convert to DataFrame
tfr500 = pd.DataFrame(records)

# mapping to standard types
tfr500['org_type'] = tfr500['Type of Organization'].map(org_type_mapping).fillna("Others")
tfr500['link_type'] = tfr500['Type of Investment'].map(link_type_mapping).fillna("Others")

# renaming and selecting variables
tfr500 = tfr500.rename(columns={'American Owner - Name': 'US Company', 
                                'Master German firm name': 'German subsidiary'})
tfr500['Vol'] = "Vol 1" +  ", p. " + tfr500['page'].astype(str)
tfr500_cleaned = tfr500[['German subsidiary', 'US Company', 'Owned Through (Allied Foreign Organization)', 'link_type', 'org_type', 'Vol', 'Type of Investment', 'Type of Organization', 'Percent Owned']]

In [672]:
tfr500_cleaned.to_csv('output/tfr_cleaned.csv', index=False)

### Tenenbaum

In [673]:
# Open the sheet
worksheet = client.open("Tenenbaum").worksheet("Sheet1 expanded")

# Get all records as list of dicts
records = worksheet.get_all_records()

# Convert to DataFrame
tenenbaum = pd.DataFrame(records)

# The boolean columns become string after conversion, so here we convert them back to booleans
bool_cols = ['mentions a subsidiary', 
             'mentions stock ownership',
             'affiliated company is AG',
             'affiliated company is GmbH',
             'mentions an affiliated company',
             'mentions a plant/office/branch', 
             'is a subsidiary of a German firm',
             'other types of agreement']
tenenbaum[bool_cols] = tenenbaum[bool_cols].apply(lambda x: x == "TRUE")

# standardized type mapping
tenenbaum['link_type'] = tenenbaum.apply(assign_link_type, axis=1)
tenenbaum['org_type'] = tenenbaum.apply(assign_org_type, axis=1)

tenenbaum['Vol'] = (
    "Vol 1" + 
    ", p. " + tenenbaum['page'].astype(str)
)

# renaming and selecting variables
tenenbaum_cleaned = tenenbaum.rename(columns={'Master US firm name': 'US Company', 
                                             'Master German firm name': 'German subsidiary',
                                             'US firm name': 'US firm (as in book)',
                                             'affiliated German firm name': 'Affiliated German firm (as in book)',
                                             'notes': 'Description'})
tenenbaum_cleaned = tenenbaum_cleaned[['German subsidiary', 'US Company', 'link_type', 'org_type', 'US firm (as in book)', 'Affiliated German firm (as in book)', 'Vol', 'Description']]

In [674]:
tenenbaum_cleaned.to_csv('output/tenenbaum_cleaned.csv', index=False)

### Moodys 32

In [675]:
# Open the sheet
worksheet = client.open("Moodys 1932").worksheet("reviewed-expanded")

# Get all records as list of dicts
records = worksheet.get_all_records()

# Convert to DataFrame
moodys32 = pd.DataFrame(records)

# The boolean columns become string after conversion, so here we convert them back to booleans
bool_cols = ['mentions a subsidiary', 
             'mentions stock ownership',
             'affiliated company is AG',
             'affiliated company is GmbH',
             'mentions an affiliated company',
             'mentions a plant/office/branch', 
             'is a subsidiary of a German firm',
             'other types of agreement']
moodys32[bool_cols] = moodys32[bool_cols].apply(lambda x: x == "TRUE")

# When manually validating the data, we filled the observations without German name with "NA". Here we replace them with empty strings
moodys32['affiliated German firm name'] = moodys32['affiliated German firm name'].replace('NA', '')

# standardized type mapping
moodys32['link_type'] = moodys32.apply(assign_link_type, axis=1)
moodys32['org_type'] = moodys32.apply(assign_org_type, axis=1)

moodys32['Vol'] = (
    "Vol 1" + 
    ", p. " + moodys32['page'].astype(str)
)

# renaming and selecting variables
moodys32_cleaned = moodys32.rename(columns={'Master US firm name': 'US Company', 
                                           'Master German firm name': 'German subsidiary',
                                           'affiliated German firm name': 'Affiliated German firm (as in book)',
                                           'US company name': 'US firm (as in book)',
                                           'notes': 'Description'})
moodys32_cleaned= moodys32_cleaned[['German subsidiary', 'US Company', 'link_type', 'org_type', 'US firm (as in book)', 'Affiliated German firm (as in book)', 'Vol', 'Description']]

In [676]:
moodys32_cleaned.to_csv('output/moodys32_cleaned.csv', index=False)

### Moodys 34

In [677]:
# Open the sheet
worksheet = client.open("Moodys 1934").worksheet("reviewed-expanded")

# Get all records as list of dicts
records = worksheet.get_all_records()

# Convert to DataFrame
moodys34 = pd.DataFrame(records)

# The boolean columns become string after conversion, so here we convert them back to booleans
bool_cols = ['mentions a subsidiary', 
             'mentions stock ownership',
             'affiliated company is AG',
             'affiliated company is GmbH',
             'mentions an affiliated company',
             'mentions a plant/office/branch', 
             'is a subsidiary of a German firm',
             'other types of agreement']
moodys34[bool_cols] = moodys34[bool_cols].apply(lambda x: x == "TRUE")

# When manually validating the data, we filled the observations without German name with "NA". Here we replace them with empty strings
moodys34['affiliated German firm name'] = moodys34['affiliated German firm name'].replace('NA', '')

# standardized type mapping
moodys34['link_type'] = moodys34.apply(assign_link_type, axis=1)
moodys34['org_type'] = moodys34.apply(assign_org_type, axis=1)

moodys34['Vol'] = (
    "Vol 1" + 
    ", p. " + moodys34['page'].astype(str)
)

# renaming and selecting variables
moodys34_cleaned = moodys34.rename(columns={'Master US firm name': 'US Company', 
                                           'Master German firm name': 'German subsidiary',
                                           'affiliated German firm name': 'Affiliated German firm (as in book)',
                                           'US company name': 'US firm (as in book)',
                                           'notes': 'Description'})
moodys34_cleaned = moodys34_cleaned[['German subsidiary', 'US Company', 'link_type', 'org_type', 'US firm (as in book)', 'Affiliated German firm (as in book)', 'Vol', 'Description']]

In [678]:
moodys34_cleaned.to_csv('output/moodys34_cleaned.csv', index=False)

### HBAG 32

In [679]:
# Open the sheet
worksheet = client.open("Handbuch 1932").worksheet("validated")

# Get all records as list of dicts
records = worksheet.get_all_records()

# Convert to DataFrame
hb1932 = pd.DataFrame(records)

# The boolean columns become string after conversion, so here we convert them back to booleans
bool_cols = ['Grossaktionär is a US comp',
             'Director attached to a US company', 
             'US location linked to a director',
             'US-sounding names', 
             'Bond issuance in the US', 
             'Other indicators are strong']
hb1932[bool_cols] = hb1932[bool_cols].apply(lambda x: x == "TRUE")

# standardized type mapping
hb1932['link_type'] = hb1932.apply(assign_link_type_HB, axis=1)
hb1932 = hb1932.assign(org_type = 'AG') # all firms are AG in HB

# renaming and selecting variables
hb1932 = hb1932.rename(columns={'Master US firm name': 'US Company', 
                                'Master name': 'German subsidiary',
                                'corrected firm name': 'Affiliated German firm (as in book)',
                                'US parent': 'US firm (as in book)',
                                })
hb1932['Vol'] = (
    "Vol " + hb1932['band'].astype(str) + 
    ", p. " + hb1932['firmname_page'].astype(str)
)
hb1932_result = hb1932[['German subsidiary', 'US Company', 'link_type', 'org_type', 'US firm (as in book)', 'Affiliated German firm (as in book)', 'Vol', 'Grossaktionär is a US comp',
                        'Director attached to a US company', 'US location linked to a director',
                        'US-sounding names', 'Bond issuance in the US', 'Other indicators',
                        'Other indicators are strong']]

# Excluding those not validated
hb1932_cleaned = hb1932_result[hb1932_result['link_type']!='Others']

In [680]:
hb1932_cleaned.to_csv('output/hb32_cleaned.csv', index=False)

### HBAG 34

In [681]:
# Open the sheet
worksheet = client.open("Handbuch 1934").worksheet("validated")

# Get all records as list of dicts
records = worksheet.get_all_records()

# Convert to DataFrame
hb1934 = pd.DataFrame(records)

# The boolean columns become string after conversion, so here we convert them back to booleans
bool_cols = ['Grossaktionär is a US comp',
             'Director attached to a US company', 
             'US location linked to a director',
             'US-sounding names', 
             'Bond issuance in the US', 
             'Other indicators are strong']
hb1934[bool_cols] = hb1934[bool_cols].apply(lambda x: x == "TRUE")

# standardized type mapping
hb1934['link_type'] = hb1934.apply(assign_link_type_HB, axis=1)
hb1934 = hb1934.assign(org_type = 'AG')

# renaming and selecting variables
hb1934 = hb1934.rename(columns={
    'Master US firm name': 'US Company', 
    'Master name': 'German subsidiary',
    'corrected firm name': 'Affiliated German firm (as in book)',
    'US parent': 'US firm (as in book)',
})
hb1934['band'] = hb1934['band']-90
hb1934['Vol'] = (
    "Vol " + hb1934['band'].astype(str) + 
    ", p. " + hb1934['page'].astype(str)
)
hb1934_result = hb1934[['German subsidiary', 'US Company', 'link_type', 'org_type', 'US firm (as in book)', 'Affiliated German firm (as in book)', 'Vol', 'Grossaktionär is a US comp',
       'Director attached to a US company', 'US location linked to a director',
       'US-sounding names', 'Bond issuance in the US', 'Other indicators',
       'Other indicators are strong']]

# Excluding those not validated
hb1934_cleaned = hb1934_result[hb1934_result['link_type']!='Others']

In [682]:
hb1934_cleaned.to_csv('output/hb34_cleaned.csv', index=False)

### HB GmbH

In [683]:
hb_gmbh = pd.read_excel('input/gmbh_validated.xlsx', sheet_name='validated')
hb_gmbh = hb_gmbh[~hb_gmbh['validated'].isin(['Z', 'ZN'])] # Z, ZN means no connection to US

# dropping duplicates: some closely related US companies are matched twice to the same company. We simply keep one.
hb_gmbh["is_duplicate"] = hb_gmbh.duplicated(subset=['Master German firm name', 'Master US firm name'])
hb_gmbh = hb_gmbh[hb_gmbh["is_duplicate"] == False]

# merging back to the source to access information
hb_gmbh_full = pd.read_csv('../Handbuch_GmbH_1932/output/firms_structured.csv')
hb_gmbh_merged = hb_gmbh.merge(hb_gmbh_full, left_on='german_name', right_on='firm_name', how='left')

# standardized type mapping
hb_gmbh_merged = hb_gmbh_merged.rename(columns={'validated': 'link_type'}).assign(org_type='GmbH')

# renaming and selecting variables
hb_gmbh_merged = hb_gmbh_merged.rename(columns={'Master US firm name': 'US Company', 
                                                'Master German firm name': 'German subsidiary',
                                                'german_name': 'Affiliated German firm (as in book)',
                                                'US_name': 'US firm (as in book)'})
hb_gmbh_merged['Vol'] = (
    "Vol 1" + 
    ", p. " + hb_gmbh_merged['page'].astype(int).astype(str)
)

hb_gmbh_cleaned = hb_gmbh_merged[[
    'German subsidiary', 'US Company', 'link_type', 'org_type', 'US firm (as in book)', 
    'Affiliated German firm (as in book)', 'other sources', 'notes', 'Vol', 'location', 'date', 'capital', 'business', 'persons', 'type'
]]

In [684]:
hb_gmbh_cleaned.to_csv('output/hb_gmbh_cleaned.csv', index=False)

## Merging all

### Subsidiaries

In [685]:
priority = {"USO": 1, "USOP": 2, "USC": 3, "USB": 4, "DEO": 5, "Others": 6}

def to_german_df(df):
    df_german = df.copy().drop(columns='US Company')
    df_german["priority"] = df_german["link_type"].map(priority)
    df_german = (
        df_german.loc[df_german.groupby("German subsidiary")["priority"].idxmin()]
        .drop(columns=["priority"])
        .reset_index(drop=True)
    )
    df_german = df_german[df_german['German subsidiary']!='']
    df_german = df_german[['German subsidiary', 'link_type', 'org_type']]
    return df_german

In [686]:
cleaned_dfs = [
    tfr500_cleaned,
    tenenbaum_cleaned,
    moodys32_cleaned,
    moodys34_cleaned,
    hb1932_cleaned, 
    hb1934_cleaned,
    hb_gmbh_cleaned
]

german_dfs = [to_german_df(df) for df in cleaned_dfs]

german_df_keys = [
    'TFR-500',
    'Tenenbaum',
    'Moodys 32',
    'Moodys 34',
    'IDed in HB 32',
    'IDed in HB 34',
    'IDed in HB GmbH'
]

In [687]:
# Initialize merged with the first df
subsidiary_merged = german_dfs[0].assign(**{german_df_keys[0]: True}).rename(columns={'link_type': f'{german_df_keys[0]}_link_type', 'org_type': f'{german_df_keys[0]}_org_type'})

# Loop through the rest
for key, df in zip(german_df_keys[1:], german_dfs[1:]):
    subsidiary_merged = subsidiary_merged.merge(
        df.assign(**{key: True}).rename(columns={'link_type': f'{key}_link_type', 'org_type': f'{key}_org_type'}),
        on="German subsidiary", how="outer"
    )

# Fill missing flags with False
flag_cols = [col for col in subsidiary_merged.columns if col in german_df_keys]
subsidiary_merged[flag_cols] = subsidiary_merged[flag_cols].fillna(False)

/var/folders/44/38ww_5ts5z53mxh7j8kt4dt40000gn/T/ipykernel_11358/1570371015.py:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  subsidiary_merged[flag_cols] = subsidiary_merged[flag_cols].fillna(False)


In [688]:
subsidiary_merged.to_csv('output/all_subsidiaries.csv', index=False)

### Searching for subsidiaries in HBs

#### Searching for subsidiaries in HB AG 32

In [689]:
# Subset once: those not in HB 32
subsidiaries_not_ided_in_hb32 = subsidiary_merged[~subsidiary_merged['IDed in HB 32']].copy()

# Collect all columns ending in _org_type except HB 32
org_type_cols = [f"{s}_org_type" for s in sources if s != "IDed in HB 32"]

# Find all subsidiaries with "AG" in any org_type column
mask = subsidiaries_not_ided_in_hb32[org_type_cols].eq("AG").any(axis=1)

# Get unique list
subsidiaries_to_search_list = sorted(subsidiaries_not_ided_in_hb32.loc[mask, "German subsidiary"].unique())

In [690]:
# hb32_searched = pd.read_excel('hb32_manual_searched.xlsx')
subsidiaries_to_search_hb32_df = pd.DataFrame({'German subsidiary': subsidiaries_to_search_list})

# importing the previously searched 
manual_searched_hb32 = pd.read_excel('manual/subsidiaries_to_search_hb32_manual.xlsx')

In [691]:
auto_manual_merged = subsidiaries_to_search_hb32_df.merge(manual_searched_hb32[['German subsidiary', 
                                                              'firmname',
                                                              'band', 
                                                              'firmname_page',
                                                              'Present in HB 32']], on='German subsidiary', how='left', indicator=True)

In [692]:
auto_manual_merged.to_excel('manual/subsidiaries_to_search_hb32_auto.xlsx', index=False)

In [693]:
missing_rows = auto_manual_merged[auto_manual_merged['Present in HB 32'].isna()]

if not missing_rows.empty:
    missing_subs = missing_rows['German subsidiary'].tolist()
    raise ValueError(
        f"Column 'Present in HB 32' contains missing values for these subsidiaries: {missing_subs}"
    )

In [694]:
# After manually completing the search
manual_searched_hb32 = pd.read_excel('manual/subsidiaries_to_search_hb32_manual.xlsx')

In [695]:
HB32 = pd.read_csv('input_handbuch/HB32.csv')

manual_searched_hb32_merged_hb32 = manual_searched_hb32.merge(HB32, on='firmname', how='left')

unmatched_rows = manual_searched_hb32_merged_hb32[manual_searched_hb32_merged_hb32['Present in HB 32']==1 & manual_searched_hb32_merged_hb32['band_y'].isna()]

if not unmatched_rows.empty:
    unmatched_subs = unmatched_rows['German subsidiary'].tolist()
    raise ValueError(
        f": Unmatched: {unmatched_subs}"
    )

manual_searched_hb32_merged_hb32 = manual_searched_hb32[['German subsidiary', 'firmname']].merge(HB32, on='firmname', how='inner')
manual_searched_hb32_merged_hb32['Vol'] = (
    "Vol " + manual_searched_hb32_merged_hb32['band'].astype(int).astype(str) + 
    ", p. " + manual_searched_hb32_merged_hb32['firmname_page'].astype(int).astype(str)
)
manual_searched_hb32_merged_hb32['org_type'] = 'AG'
manual_searched_hb32_merged_hb32 = manual_searched_hb32_merged_hb32.rename(columns={'firmname': 'Affiliated German firm (as in book)'})
manual_searched_hb32_merged_hb32 = manual_searched_hb32_merged_hb32[['German subsidiary', 'Affiliated German firm (as in book)', 'Vol']]

hb1932_cleaned_expanded = pd.concat([hb1932_cleaned, manual_searched_hb32_merged_hb32])

In [696]:
hb1932_cleaned_expanded.to_csv('output/hb1932_cleaned_expanded.csv', index=False)

#### Searching for subsidiaries in HB AG 34

In [697]:
# Subset once: those not in HB 32
subsidiaries_not_ided_in_hb34 = subsidiary_merged[~subsidiary_merged['IDed in HB 34']].copy()

# Collect all columns ending in _org_type except HB 34
org_type_cols = [f"{s}_org_type" for s in sources if s != "IDed in HB 34"]

# Find all subsidiaries with "AG" in any org_type column
mask = subsidiaries_not_ided_in_hb34[org_type_cols].eq("AG").any(axis=1)

# Get unique list
subsidiaries_to_search_list = sorted(subsidiaries_not_ided_in_hb34.loc[mask, "German subsidiary"].unique())

In [698]:
subsidiaries_to_search_hb34_df = pd.DataFrame({'German subsidiary': subsidiaries_to_search_list})

# importing the previously searched 
manual_searched_hb34 = pd.read_excel('manual/subsidiaries_to_search_hb34_manual.xlsx')
# manual_searched_hb34 = pd.read_excel('hb34_manual_searched.xlsx')

In [699]:
auto_manual_merged = subsidiaries_to_search_hb34_df.merge(manual_searched_hb34[['German subsidiary', 
                                                              'firmname',
                                                              'band', 
                                                              'firmname_page',
                                                              'Present in HB 34'
                                                              ]], on='German subsidiary', how='left', indicator=True)

In [700]:
auto_manual_merged.to_excel('manual/subsidiaries_to_search_hb34_auto.xlsx', index=False)

In [701]:
missing_rows = auto_manual_merged[auto_manual_merged['Present in HB 34'].isna()]

if not missing_rows.empty:
    missing_subs = missing_rows['German subsidiary'].tolist()
    raise ValueError(
        f"Column 'Present in HB 34' contains missing values for these subsidiaries: {missing_subs}"
    )

In [702]:
# After manually completing the search
manual_searched_hb34 = pd.read_excel('manual/subsidiaries_to_search_hb34_manual.xlsx')
manual_searched_hb34['firmname'] = manual_searched_hb34['firmname'].str.strip()

In [703]:
manual_searched_hb34

,German subsidiary,firmname,band,firmname_page,Present in HB 34,_merge
0,"""Borvisk"" Kunstseiden A. G.",„Borvisk“ Kunstseiden-Aktiengesellschaft.,4.0,746.0,1,both
1,"""Debag"" Betriebstoff A.G.",„Debag“ Deutsche Betriebsstoff-Aktiengesellsch...,3.0,1373.0,1,both
2,"""Dom"" Finanz A. G. für chemische Unternehmenungen",„Dom“ Finanz-Aktiengesellschaft für chem. Unte...,3.0,1242.0,1,both
3,"""Rustica"" A.G. fuer Grunderwerb",„Rustica“ Aktiengesellschaft für Grunderwerh.,3.0,498.0,1,both
4,A. S. Hinds AG,Hinds Aktiengesellschaft.,3.0,902.0,1,both
...,...,...,...,...,...,...
123,Vereinigte Bayerische Telephonwerke AG,Vereinigte Bayerische Telephonwerke Aktiengese...,2.0,1549.0,1,both
124,Vereinigte Berliner Kohlenhandel A.G.,Vereinigte Berliner Kohlenhändler-Akt.-Ges.,1.0,1417.0,1,both
125,Vereinigte Glanzstoff A.G.,Vereinigte Glanzstoff-Fabriken A.-G. zu Elberf...,3.0,389.0,1,both
126,Verkehrs & Handels Aktiengesellschaft,Verkehrs- und Handels-Aktiengesellschaft.,1.0,334.0,1,both


In [704]:
HB34 = pd.read_csv('input_handbuch/HB34.csv').rename(columns={'page': 'firmname_page'})

manual_searched_hb34_merged_hb34 = manual_searched_hb34.merge(HB34, on='firmname', how='left')

unmatched_rows = manual_searched_hb34_merged_hb34[manual_searched_hb34_merged_hb34['Present in HB 34']==1 & manual_searched_hb34_merged_hb34['band_y'].isna()]

if not unmatched_rows.empty:
    unmatched_subs = unmatched_rows['German subsidiary'].tolist()
    raise ValueError(
        f": Unmatched: {unmatched_subs}"
    )

manual_searched_hb34_merged_hb34 = manual_searched_hb34[['German subsidiary', 'firmname']].merge(HB34, on='firmname', how='inner')
manual_searched_hb34_merged_hb34['Vol'] = (
    "Vol " + manual_searched_hb34_merged_hb34['band'].astype(int).astype(str)+ 
    ", p. " + manual_searched_hb34_merged_hb34['firmname_page'].astype(int).astype(str)
)
manual_searched_hb34_merged_hb34['org_type'] = 'AG'
manual_searched_hb34_merged_hb34 = manual_searched_hb34_merged_hb34.rename(columns={'firmname': 'Affiliated German firm (as in book)'})
manual_searched_hb34_merged_hb34 = manual_searched_hb34_merged_hb34[['German subsidiary', 'org_type', 'Affiliated German firm (as in book)', 'Vol']]

# hb1934_cleaned_expanded = pd.concat([hb1934_cleaned, manual_searched_hb34_merged_hb34])

In [705]:
hb1934_cleaned

,German subsidiary,US Company,link_type,org_type,US firm (as in book),Affiliated German firm (as in book),Vol,Grossaktionär is a US comp,Director attached to a US company,US location linked to a director,US-sounding names,Bond issuance in the US,Other indicators,Other indicators are strong
3,Vereinigte Königs- und Laurahütte,"W. A. Harriman & Co., Inc., New York.",USOP,AG,W. A. Harriman & Co.,"Vereinigte Königs- und Laurahütte, Actien-Gese...","Vol 1, p. 331",False,True,True,False,False,,False
4,,,USB,AG,,"Gutehoffnungshütte, Aktienverein für Bergbau u...","Vol 1, p. 361",False,False,False,False,True,,False
5,Deutsche Babcock & Wilcox Dampfkessel-Werke AG,The Babcock & Wilcox Company,USOP,AG,Babcock & Wilcox,"Deutsche Babcock & Wilcox-Dampfkessel-Werke, A...","Vol 1, p. 377",False,False,False,False,False,"Mentions '[Babcock & Wilcox Ltd.]', which is a...",True
6,Metallisator Berlin Akt.-Ges.,Metals Coating Company of America,USO,AG,Metals Coating Company of America in Philadelphia,Metallisator Berlin Aktiengesellschaft.,"Vol 1, p. 390",True,False,True,False,False,,False
7,Adam Opel Aktiengesellschaft,General Motors Corporation (Overseas Operations),USO,AG,General Motors,Adam Opel Aktiengesellschaft.,"Vol 1, p. 481",True,True,True,True,False,,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,Berliner Städtische Elektrizitätswerke Akt.-Ge...,"Harris, Forbes & Co.",USOP,AG,"Chase Harris Forbes Corporation, New York",Berliner Städtische Elektrizitätswerke Akt.-Ge...,"Vol 4, p. 1015",False,True,True,False,True,,False
120,Hirsch-Bräu Aktiengesellschaft.,,USC,AG,,Hirsch-Bräu Aktiengesellschaft.,"Vol 4, p. 1082",False,False,True,False,False,,False
121,"Ha-No-Mex, Handels-Akt.-Ges.",,USC,AG,,"Ha-No-Mex, Handels-Aktiengesellschaft in Liqu.","Vol 4, p. 1141",False,False,True,False,False,,False
122,National-Film Verleih- u. Vertriebs-Aktiengese...,,USOP,AG,,National-Film Verleih- u. Vertriebs-Aktiengese...,"Vol 4, p. 1164",False,False,False,False,False,Mentions an 'amerik. Rechtsanw. ' on the board,True


In [706]:
hb1934_cleaned_expanded

,German subsidiary,US Company,link_type,org_type,US firm (as in book),Affiliated German firm (as in book),Vol,Grossaktionär is a US comp,Director attached to a US company,US location linked to a director,US-sounding names,Bond issuance in the US,Other indicators,Other indicators are strong
3,Vereinigte Königs- und Laurahütte,"W. A. Harriman & Co., Inc., New York.",USOP,AG,W. A. Harriman & Co.,"Vereinigte Königs- und Laurahütte, Actien-Gese...","Vol 91, p. 331",False,True,True,False,False,,False
4,,,USB,AG,,"Gutehoffnungshütte, Aktienverein für Bergbau u...","Vol 91, p. 361",False,False,False,False,True,,False
5,Deutsche Babcock & Wilcox Dampfkessel-Werke AG,The Babcock & Wilcox Company,USOP,AG,Babcock & Wilcox,"Deutsche Babcock & Wilcox-Dampfkessel-Werke, A...","Vol 91, p. 377",False,False,False,False,False,"Mentions '[Babcock & Wilcox Ltd.]', which is a...",True
6,Metallisator Berlin Akt.-Ges.,Metals Coating Company of America,USO,AG,Metals Coating Company of America in Philadelphia,Metallisator Berlin Aktiengesellschaft.,"Vol 91, p. 390",True,False,True,False,False,,False
7,Adam Opel Aktiengesellschaft,General Motors Corporation (Overseas Operations),USO,AG,General Motors,Adam Opel Aktiengesellschaft.,"Vol 91, p. 481",True,True,True,True,False,,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,Vereinigte Bayerische Telephonwerke AG,NaN,NaN,AG,NaN,Vereinigte Bayerische Telephonwerke Aktiengese...,"Vol 2, p. 1549",NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,Vereinigte Berliner Kohlenhandel A.G.,NaN,NaN,AG,NaN,Vereinigte Berliner Kohlenhändler-Akt.-Ges.,"Vol 3, p. 1417",NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,Vereinigte Glanzstoff A.G.,NaN,NaN,AG,NaN,Vereinigte Glanzstoff-Fabriken A.-G. zu Elberf...,"Vol 3, p. 389",NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,Verkehrs & Handels Aktiengesellschaft,NaN,NaN,AG,NaN,Verkehrs- und Handels-Aktiengesellschaft.,"Vol 1, p. 334",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [707]:
hb1934_cleaned_expanded.to_csv('output/hb1934_cleaned_expanded.csv', index=False)

#### Searching for subsidiaries in HB GmbH

In [708]:
# Subset once: those not in HB GmbH
subsidiaries_not_ided_in_hb_gmbh = subsidiary_merged[~subsidiary_merged['IDed in HB GmbH']].copy()

# Collect all columns ending in _org_type except HB 34
org_type_cols = [f"{s}_org_type" for s in sources if s != "IDed in HB GmbH"]

# Find all subsidiaries with "AG" in any org_type column
mask = subsidiaries_not_ided_in_hb_gmbh[org_type_cols].eq("GmbH").any(axis=1)

# Get unique list
subsidiaries_to_search_list = sorted(subsidiaries_not_ided_in_hb_gmbh.loc[mask, "German subsidiary"].unique())

In [709]:
subsidiaries_to_search_hb_gmbh_df = pd.DataFrame({'German subsidiary': subsidiaries_to_search_list})

# importing the previously searched 
# manual_searched_hb_gmbh = pd.read_csv('list_gmbh/output/all_searched_in_hb_gmbh.csv')
manual_searched_hb_gmbh = pd.read_excel('manual/subsidiaries_to_search_hb_gmbh_manual.xlsx')

In [710]:
auto_manual_merged = subsidiaries_to_search_hb_gmbh_df.merge(manual_searched_hb_gmbh[['German subsidiary', 
                                                              'firm_name',
                                                              'Present in HB GmbH']], on='German subsidiary', how='left', indicator=True)

In [711]:
auto_manual_merged.to_excel('manual/subsidiaries_to_search_hb_gmbh_auto.xlsx', index=False)

In [712]:
if auto_manual_merged['Present in HB GmbH'].isna().any():
    raise ValueError(f"Column 'Present in HB GmbH' contains missing values.")

In [713]:
# After manually completing the search
manual_searched_hb_gmbh = pd.read_excel('manual/subsidiaries_to_search_hb_gmbh_manual.xlsx')

In [714]:
HB_GmbH = pd.read_csv('input_handbuch/HB_GmbH_32.csv')[['firm_name', 'page', 'location', 'date', 'capital', 'business', 'persons', 'type']]
HB_GmbH = HB_GmbH[HB_GmbH['firm_name'].notna()]

manual_searched_hb_gmbh_merged_hb_gmbh = manual_searched_hb_gmbh.merge(HB_GmbH, on='firm_name', how='left')

unmatched_rows = manual_searched_hb_gmbh_merged_hb_gmbh[manual_searched_hb_gmbh_merged_hb_gmbh['Present in HB GmbH']==1 & manual_searched_hb_gmbh_merged_hb_gmbh['location'].isna()]

if not unmatched_rows.empty:
    unmatched_subs = unmatched_rows['firm_name'].tolist()
    raise ValueError(
        f": Unmatched: {unmatched_subs}"
    )

manual_searched_hb_gmbh_merged_hb_gmbh = manual_searched_hb_gmbh.merge(HB_GmbH, on='firm_name', how='inner')
manual_searched_hb_gmbh_merged_hb_gmbh['Vol'] = (
    "Vol 1" + 
    ", p. " + manual_searched_hb_gmbh_merged_hb_gmbh['page'].astype(int).astype(str)
)
manual_searched_hb_gmbh_merged_hb_gmbh['org_type'] = 'GmbH'
manual_searched_hb_gmbh_merged_hb_gmbh = manual_searched_hb_gmbh_merged_hb_gmbh.rename(columns={'firm_name': 'Affiliated German firm (as in book)'})
manual_searched_hb_gmbh_merged_hb_gmbh = manual_searched_hb_gmbh_merged_hb_gmbh[['German subsidiary', 'org_type', 'Affiliated German firm (as in book)', 'Vol', 'location', 'date', 'capital', 'business', 'persons', 'type']]

hb_gmbh_cleaned_expanded = pd.concat([hb_gmbh_cleaned, manual_searched_hb_gmbh_merged_hb_gmbh])

In [715]:
hb_gmbh_cleaned_expanded.to_csv('output/hb_gmbh_cleaned_expanded.csv', index=False)

### Merging all searched

In [716]:
# Initialize merged with the first df
subsidiary_merged = german_dfs[0].assign(**{german_df_keys[0]: True}).rename(columns={'link_type': f'{german_df_keys[0]}_link_type', 'org_type': f'{german_df_keys[0]}_org_type'})

# Loop through the rest
for key, df in zip(german_df_keys[1:], german_dfs[1:]):
    subsidiary_merged = subsidiary_merged.merge(
        df.assign(**{key: True}).rename(columns={'link_type': f'{key}_link_type', 'org_type': f'{key}_org_type'}),
        on="German subsidiary", how="outer"
    )

# Fill missing flags with False
flag_cols = [col for col in subsidiary_merged.columns if col in german_df_keys]
subsidiary_merged[flag_cols] = subsidiary_merged[flag_cols].fillna(False)

/var/folders/44/38ww_5ts5z53mxh7j8kt4dt40000gn/T/ipykernel_11358/1570371015.py:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  subsidiary_merged[flag_cols] = subsidiary_merged[flag_cols].fillna(False)


In [717]:
def clean_searched_df(df, source):
    df_cleaned = df.rename(columns={'Master German firm name': 'German subsidiary'})
    df_cleaned = df_cleaned[['German subsidiary', f'Present in {source}']]
    df_cleaned[f'Present in {source}'] = df_cleaned[f'Present in {source}'].astype(bool)
    return df_cleaned

In [718]:
searched_dfs = [
    manual_searched_hb32, 
    manual_searched_hb34,
    manual_searched_hb_gmbh
]

searched_dfs_keys = [
    'HB 32',
    'HB 34',
    'HB GmbH'
]

cleaned_searched_dfs = [clean_searched_df(df, source) for df, source in zip(searched_dfs, searched_dfs_keys)]

In [719]:
# Initialize merged with the first df
subsidiary_merged = german_dfs[0].assign(**{german_df_keys[0]: True}).rename(columns={'link_type': f'{german_df_keys[0]}_link_type', 'org_type': f'{german_df_keys[0]}_org_type'})

# Loop through the rest
for key, df in zip(german_df_keys[1:], german_dfs[1:]):
    subsidiary_merged = subsidiary_merged.merge(
        df.assign(**{key: True}).rename(columns={'link_type': f'{key}_link_type', 'org_type': f'{key}_org_type'}),
        on="German subsidiary", how="outer"
    )

# Fill missing flags with False
flag_cols = [col for col in subsidiary_merged.columns if col in german_df_keys]
subsidiary_merged[flag_cols] = subsidiary_merged[flag_cols].fillna(False)

/var/folders/44/38ww_5ts5z53mxh7j8kt4dt40000gn/T/ipykernel_11358/1570371015.py:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  subsidiary_merged[flag_cols] = subsidiary_merged[flag_cols].fillna(False)


In [720]:
subsidiary_merged_searched = subsidiary_merged.merge(cleaned_searched_dfs[0], on='German subsidiary', how='left')
subsidiary_merged_searched['Present in HB 32'] = subsidiary_merged_searched['Present in HB 32'].fillna(subsidiary_merged_searched['IDed in HB 32'])

# Loop through the rest
for key, df in zip(searched_dfs_keys[1:], cleaned_searched_dfs[1:]):
    subsidiary_merged_searched = subsidiary_merged_searched.merge(
        df, on="German subsidiary", how="left"
    )
    subsidiary_merged_searched[f'Present in {key}'] = subsidiary_merged_searched[f'Present in {key}'].fillna(subsidiary_merged_searched[f'IDed in {key}'])


subsidiary_merged_searched

/var/folders/44/38ww_5ts5z53mxh7j8kt4dt40000gn/T/ipykernel_11358/890401275.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  subsidiary_merged_searched['Present in HB 32'] = subsidiary_merged_searched['Present in HB 32'].fillna(subsidiary_merged_searched['IDed in HB 32'])
/var/folders/44/38ww_5ts5z53mxh7j8kt4dt40000gn/T/ipykernel_11358/890401275.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  subsidiary_merged_searched[f'Present in {key}'] = subsidiary_merged_searched[f'Present in {key}'].fillna(subsidiary_merged_searched[f'IDed in {key}'])
/var/fo

,German subsidiary,TFR-500_link_type,TFR-500_org_type,TFR-500,Tenenbaum_link_type,Tenenbaum_org_type,Tenenbaum,Moodys 32_link_type,Moodys 32_org_type,Moodys 32,...,IDed in HB 32,IDed in HB 34_link_type,IDed in HB 34_org_type,IDed in HB 34,IDed in HB GmbH_link_type,IDed in HB GmbH_org_type,IDed in HB GmbH,Present in HB 32,Present in HB 34,Present in HB GmbH
0,"""Borvisk"" Kunstseiden A. G.",USO,AG,True,NaN,NaN,False,NaN,NaN,False,...,False,NaN,NaN,False,NaN,NaN,False,True,True,False
1,"""Borvisk"" Kunstseiden-Vertriebsgesellschaft, m...",USO,GmbH,True,NaN,NaN,False,NaN,NaN,False,...,False,NaN,NaN,False,NaN,NaN,False,False,False,True
2,"""Debag"" Betriebstoff A.G.",NaN,NaN,False,USO,AG,True,NaN,NaN,False,...,False,NaN,NaN,False,NaN,NaN,False,True,True,False
3,"""Dom"" Finanz A. G. für chemische Unternehmenungen",NaN,NaN,False,USO,AG,True,NaN,NaN,False,...,False,NaN,NaN,False,NaN,NaN,False,True,True,False
4,"""Gluckauf"" Deutsche Oel Gesellschaft Hersfeld ...",USO,GmbH,True,NaN,NaN,False,NaN,NaN,False,...,False,NaN,NaN,False,NaN,NaN,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,Zoellner Werke G.m.b.H.,USO,GmbH,True,NaN,NaN,False,NaN,NaN,False,...,False,NaN,NaN,False,NaN,NaN,False,False,False,False
518,Zuckerrubensaftfabrik,USO,GmbH,True,NaN,NaN,False,NaN,NaN,False,...,False,NaN,NaN,False,NaN,NaN,False,False,False,True
519,Zukunft Lebensversicherungsbank Aktiengesellsc...,NaN,NaN,False,NaN,NaN,False,NaN,NaN,False,...,False,USOP,AG,True,NaN,NaN,False,True,True,False
520,missing,USO,Others,True,NaN,NaN,False,NaN,NaN,False,...,False,NaN,NaN,False,NaN,NaN,False,False,False,False


In [721]:
subsidiary_merged_searched.to_csv('output/all_subsidiaries_searched.csv', index=False)

In [722]:
bands = [
    '1. Band 1-1648',
    '2. Band 1649-3472',
    '3. Band 3473-5104',
    '4. Band 5105-6759'
]

hb32_full = pd.concat([pd.read_csv(f'input_handbuch/HB32_firm_names/firm_names_{band}.csv').assign(band=str(i+1)) for i, band in enumerate(bands)])
hb32_full = hb32_full[['band', 'firmname_page', 'firmname']]
hb32_full = hb32_full.rename({'band': 'vol', 'firmname_page': 'page'})
hb32_full['firmname'] = hb32_full['firmname'].str.strip().str.replace(r'\s+', ' ', regex=True).str.strip()
hb32_full.to_csv('input_handbuch/HB32.csv', index=False)

In [723]:
bands = [91, 92, 93, 94]

hb34_full = pd.concat([pd.read_csv(f'input_handbuch/HB34_firm_names/firmnames_band{band}.csv').assign(band=str(i+1)) for i, band in enumerate(bands)])
hb34_full = hb34_full[['band', 'page', 'firmname']]
hb34_full = hb34_full.rename({'band': 'vol'})
hb34_full['firmname'] = hb34_full['firmname'].str.strip().str.replace(r'\s+', ' ', regex=True).str.strip()
hb34_full.to_csv('input_handbuch/HB34.csv', index=False)

In [724]:
presence_source_dict = {
    'TFR-500': 'TFR-500',
    'Tenenbaum': 'Tenenbaum',
    'Moodys 32': 'Moodys 32',
    'Moodys 34': 'Moodys 34',
    'Present in HB 32': 'HB 32',
    'Present in HB 34': 'HB 34',
    'Present in HB GmbH': 'HB GmbH'
}

### Merging parents

In [725]:
def to_parent_df(df):
    df_parent = df.copy().drop(columns=['German subsidiary', 'org_type'])
    df_parent = df_parent[df_parent['US Company'].notna()]
    df_parent["priority"] = df_parent["link_type"].map(priority)
    df_parent = (
        df_parent.loc[df_parent.groupby("US Company")["priority"].idxmin()]
        .drop(columns=["priority"])
        .reset_index(drop=True)
    )
    df_parent = df_parent[df_parent['US Company']!='']
    df_parent = df_parent[['US Company', 'link_type']]
    return df_parent

In [726]:
cleaned_dfs = [
    tfr500_cleaned,
    tenenbaum_cleaned,
    moodys32_cleaned,
    moodys34_cleaned,
    hb1932_cleaned, 
    hb1934_cleaned,
    hb_gmbh_cleaned
]

parent_dfs = [to_parent_df(df) for df in cleaned_dfs]

parent_df_keys = [
    'TFR-500',
    'Tenenbaum',
    'Moodys 32',
    'Moodys 34',
    'HB 32',
    'HB 34',
    'HB GmbH'
]


In [727]:
# Initialize merged with the first df
parent_merged = parent_dfs[0].assign(**{parent_df_keys[0]: True}).rename(columns={'link_type': f'{parent_df_keys[0]}_link_type'})

# Loop through the rest
for key, df in zip(parent_df_keys[1:], parent_dfs[1:]):
    parent_merged = parent_merged.merge(
        df.assign(**{key: True}).rename(columns={'link_type': f'{key}_link_type'}),
        on="US Company", how="outer"
    )

# Fill missing flags with False
flag_cols = [col for col in parent_merged.columns if col in parent_df_keys]
parent_merged[flag_cols] = parent_merged[flag_cols].fillna(False)

/var/folders/44/38ww_5ts5z53mxh7j8kt4dt40000gn/T/ipykernel_11358/284979574.py:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  parent_merged[flag_cols] = parent_merged[flag_cols].fillna(False)


In [728]:
parent_merged.to_csv('output/all_parents.csv', index=False)